In [1]:
import torch
import pickle
import numpy as np
from argparse import Namespace
from torch.utils.data import DataLoader
import torch.nn.functional as F
from heterogt.utils.tokenizer import EHRTokenizer
from heterogt.utils.dataset import FineTuneEHRDataset, batcher, expand_level3
from heterogt.utils.train import train_with_early_stopping, PHENO_ORDER
from heterogt.utils.seed import set_random_seed
from heterogt.model.model import HeteroGTFineTune

Disabling PyTorch because PyTorch >= 2.1 is required but found 1.13.1
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [2]:
set_random_seed(123)

[INFO] Random seed set to 123


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [4]:
config = Namespace(
    dataset = "MIMIC-III",
    tasks = ["death", "readmission", "stay", "next_diag_6m", "next_diag_12m"],
    task_index = 4,  # index of the task to train
    token_type = ["diag", "med", "lab", "pro"],
    special_tokens = ["[PAD]", "[CLS]"],
    # only for tf layer
    attn_mask_dicts = [{1:[6,7], 2:[6,7], 3:[6,7], 4:[6,7], 5:[6,7], 6:[1,2,3,4,5,6,7], 7:[1,2,3,4,5,6,7]}, 
                       {1:[6,7], 2:[6,7], 3:[6,7], 4:[6,7], 5:[6,7], 6:[1,2,3,4,5,6,7], 7:[1,2,3,4,5,6,7]}], 
    d_model = 64,
    num_heads = 4,
    batch_size = 32,
    lr = 1e-3,
    epochs = 500,
    early_stop_patience = 5,
    group_code_thre = 5,  # if there are group_code_thre diag codes belongs to the same group ICD code, then the group code is generated
    use_pretrained_model = True,
    pretrain_mask_rate = 0.7,
    pretrain_cls_ontology_weight = 5e-2,
    pretrain_visit_ontology_weight = 5e-2,
    pretrain_adm_type_weight = 0,
    dec_loss_lambda = 1e-2,
)

In [5]:
full_data_path = f"./data_process/{config.dataset}-processed/mimic.pkl"  # for tokenizer
curr_task = config.tasks[config.task_index]
print("Current task:", curr_task)
if curr_task == "next_diag_6m":
    finetune_data_path = f"./data_process/{config.dataset}-processed/mimic_nextdiag_6m.pkl"
elif curr_task == "next_diag_12m":
    finetune_data_path = f"./data_process/{config.dataset}-processed/mimic_nextdiag_12m.pkl"
else:
    finetune_data_path = f"./data_process/{config.dataset}-processed/mimic_downstream.pkl"

Current task: next_diag_12m


In [6]:
ehr_full_data = pickle.load(open(full_data_path, 'rb'))
group_code_sentences = [expand_level3()[1]]
diag_sentences = ehr_full_data["ICD9_CODE"].values.tolist()
med_sentences = ehr_full_data["NDC"].values.tolist()
lab_sentences = ehr_full_data["LAB_TEST"].values.tolist()
pro_sentences = ehr_full_data["PRO_CODE"].values.tolist()
age_sentences = [[str(c)] for c in set(ehr_full_data["AGE"].values.tolist())] # important of [[]]
adm_type_sentences = ehr_full_data["ADMISSION_TYPE"].values.tolist()
max_admissions = ehr_full_data.groupby("SUBJECT_ID")["HADM_ID"].nunique().max()
config.max_num_adms = max_admissions
print(f"Max admissions per patient: {config.max_num_adms}")

Max admissions per patient: 8


In [7]:
tokenizer = EHRTokenizer(age_sentences, group_code_sentences, diag_sentences, med_sentences, lab_sentences, 
                         pro_sentences, special_tokens=config.special_tokens, adm_types_sentences=adm_type_sentences)
config.label_vocab_size = len(PHENO_ORDER)  # a set of predifined diseases
config.global_vocab_size = len(tokenizer.vocab.id2word)
config.age_vocab_size = tokenizer.token_number("age")
config.group_code_vocab_size = tokenizer.token_number("group")
print(f"Age vocabulary size: {config.age_vocab_size}")
print(f"Group code vocabulary size: {config.group_code_vocab_size}")

Age vocabulary size: 18
Group code vocabulary size: 19


In [8]:
train_data, val_data, test_data = pickle.load(open(finetune_data_path, 'rb'))
# example label percentage
print("Percentage of DEATH in test dataset:",
      (test_data["DEATH"] == True).mean() * 100, "%")

print("Percentage of READMISSION in test dataset:",
      (test_data["READMISSION"] == 1).mean() * 100, "%")

print("Percentage of STAY>7 days in test dataset:",
      (test_data["STAY_DAYS"] > 7).mean() * 100, "%")

Percentage of DEATH in test dataset: 8.114199849737041 %
Percentage of READMISSION in test dataset: 64.08715251690458 %
Percentage of STAY>7 days in test dataset: 55.10894064613073 %


In [9]:
train_dataset = FineTuneEHRDataset(train_data, tokenizer, token_type=config.token_type, task=curr_task, 
                                   max_num_adms=config.max_num_adms, group_code_thre=config.group_code_thre)
val_dataset = FineTuneEHRDataset(val_data, tokenizer, token_type=config.token_type, task=curr_task, 
                                 max_num_adms=config.max_num_adms, group_code_thre=config.group_code_thre)
test_dataset = FineTuneEHRDataset(test_data, tokenizer, token_type=config.token_type, task=curr_task, 
                                   max_num_adms=config.max_num_adms, group_code_thre=config.group_code_thre)

In [10]:
long_adm_seq_crite = 3
val_long_seq_idx, test_long_seq_idx = [], []
for i in range(len(val_dataset)):
    hadm_id = list(val_dataset.records.keys())[i]
    num_adms = len(val_dataset.records[hadm_id])
    if num_adms >= long_adm_seq_crite:
        val_long_seq_idx.append(i)
for i in range(len(test_dataset)):
    hadm_id = list(test_dataset.records.keys())[i]
    num_adms = len(test_dataset.records[hadm_id])
    if num_adms >= long_adm_seq_crite:
        test_long_seq_idx.append(i)
print(len(val_long_seq_idx), len(test_long_seq_idx))

208 186


In [11]:
num_group_code = []
for i in range(len(train_dataset)):
    input_ids, token_types, adm_index, age_ids, diag_group_codes, labels = train_dataset[i]
    count = (token_types[0] == 6).sum().item()
    num_group_code.append(count)
print("Mean group token numer per patient", np.mean(num_group_code))

Mean group token numer per patient 0.8268720127456187


In [12]:
train_dataloader = DataLoader(
    train_dataset,
    collate_fn=batcher(tokenizer, n_token_type=len(config.token_type), is_pretrain = False),
    shuffle=True,
    batch_size=config.batch_size,
)

val_dataloader = DataLoader(
    val_dataset,
    collate_fn=batcher(tokenizer, n_token_type=len(config.token_type), is_pretrain = False),
    shuffle=False,
    batch_size=config.batch_size,
)

test_dataloader = DataLoader(
    test_dataset,
    collate_fn=batcher(tokenizer, n_token_type=len(config.token_type), is_pretrain = False),
    shuffle=False,
    batch_size=config.batch_size,
)

In [13]:
for batch in train_dataloader:
    pass  # just to check if the dataloader works
for batch in val_dataloader:
    pass  # just to check if the dataloader works
for batch in test_dataloader:
    pass  # just to check if the dataloader works
print("All pass!")

All pass!


In [14]:
if curr_task in ["death", "stay", "readmission"]:
    eval_metric = "f1"
    task_type = "binary"
    loss_fn = F.binary_cross_entropy_with_logits
else:
    eval_metric = "f1"
    task_type = "l2r"
    loss_fn = lambda x, y: F.binary_cross_entropy_with_logits(x, y)

In [15]:
input_ids, token_types, adm_index, age_ids, diag_code_group_dicts, labels = next(iter(train_dataloader))
print("Input IDs shape:", input_ids.shape)
print("Token Types shape:", token_types.shape)
print("Admission Index shape:", adm_index.shape)
print("Age IDs shape:", age_ids.shape)
print("Diag Code Group Dict number:", len(diag_code_group_dicts))
print("Labels shape:", labels.shape)

Input IDs shape: torch.Size([32, 193])
Token Types shape: torch.Size([32, 193])
Admission Index shape: torch.Size([32, 193])
Age IDs shape: torch.Size([32, 4])
Diag Code Group Dict number: 32
Labels shape: torch.Size([32, 18])


# Model Walkthrough

In [16]:
# load pretrained model
if config.use_pretrained_model:
    pretrain_exp_name = (
    f"{config.dataset}-{config.pretrain_mask_rate}-{config.d_model}-{config.pretrain_cls_ontology_weight}-{config.pretrain_visit_ontology_weight}-{config.pretrain_adm_type_weight}"
)
    print(pretrain_exp_name)
    save_path = "./pretrained_models/" + pretrain_exp_name
    state_dict = torch.load(f"{save_path}/pretrained_model.pt", map_location="cpu")

MIMIC-III-0.7-64-0.05-0.05-0


In [17]:
final_metrics, final_long_seq_metrics = [], []
for i in range(15):
    print(f"================================{i+1}==================================")
    model = HeteroGTFineTune(tokenizer=tokenizer, token_types=config.token_type, d_model=config.d_model, num_heads=config.num_heads, layer_types=['gnn', 'tf', 'gnn', 'tf'], max_num_adms=config.max_num_adms, 
                     device=device, task=curr_task, label_vocab_size=config.label_vocab_size, attn_mask_dicts=config.attn_mask_dicts,
                     use_cls_cat=True).to(device)
    if config.use_pretrained_model:
        model.load_weight(state_dict)
    optimizer = torch.optim.AdamW(model.parameters(), lr=config.lr)
    best_test_metric, best_test_long_seq_metric = train_with_early_stopping(model, train_dataloader, val_dataloader, test_dataloader,
                                                                            optimizer, loss_fn, device, config.early_stop_patience, task_type, config.epochs, 
                                                                            dec_loss_lambda=config.dec_loss_lambda, 
                                                                            val_long_seq_idx=val_long_seq_idx, test_long_seq_idx=test_long_seq_idx, 
                                                                            eval_metric=eval_metric, return_model=False)
    final_metrics.append(best_test_metric)
    final_long_seq_metrics.append(best_test_long_seq_metric)

================================1==================================


Running inference: 100%|██████████| 45/45 [00:02<00:00, 21.84it/s]


                                           precision   recall       f1  \
Acute and unspecified renal failure          40.6452  14.5161  21.3922   
Acute cerebrovascular disease                 0.0000   0.0000   0.0000   
Acute myocardial infarction                   0.0000   0.0000   0.0000   
Cardiac dysrhythmias                         62.4390  46.7153  53.4447   
Chronic kidney disease                       97.7778  12.9412  22.8571   
Chronic obstructive pulmonary disease         0.0000   0.0000   0.0000   
Conduction disorders                          0.0000   0.0000   0.0000   
Congestive heart failure; nonhypertensive    72.1604  60.0000  65.5207   
Coronary atherosclerosis and related         67.6471  13.4111  22.3844   
Disorders of lipid metabolism                50.0000   0.2625   0.5222   
Essential hypertension                       60.0000  21.5534  31.7143   
Fluid and electrolyte disorders              53.2258  11.4983  18.9112   
Gastrointestinal hemorrhage           

Running inference: 100%|██████████| 45/45 [00:02<00:00, 21.75it/s]


                                           precision   recall       f1  \
Acute and unspecified renal failure          38.9937  14.5199  21.1604   
Acute cerebrovascular disease                 0.0000   0.0000   0.0000   
Acute myocardial infarction                   0.0000   0.0000   0.0000   
Cardiac dysrhythmias                         58.7179  42.8839  49.5671   
Chronic kidney disease                       85.0000  15.5015  26.2211   
Chronic obstructive pulmonary disease         0.0000   0.0000   0.0000   
Conduction disorders                          0.0000   0.0000   0.0000   
Congestive heart failure; nonhypertensive    72.1393  54.1045  61.8337   
Coronary atherosclerosis and related         68.4211  12.1495  20.6349   
Disorders of lipid metabolism                 0.0000   0.0000   0.0000   
Essential hypertension                       51.2195  20.8333  29.6192   
Fluid and electrolyte disorders              48.8000  10.9123  17.8363   
Gastrointestinal hemorrhage           

...

                                           precision   recall       f1  \
Acute and unspecified renal failure          45.3125  47.5410  46.4000   
Acute cerebrovascular disease                37.5000  12.5000  18.7500   
Acute myocardial infarction                  40.0000   4.0816   7.4074   
Cardiac dysrhythmias                         61.5137  71.5356  66.1472   
Chronic kidney disease                       65.9942  69.6049  67.7515   
Chronic obstructive pulmonary disease        51.8717  57.0588  54.3417   
Conduction disorders                          0.0000   0.0000   0.0000   
Congestive heart failure; nonhypertensive    65.0000  67.9104  66.4234   
Coronary atherosclerosis and related         61.0909  52.3364  56.3758   
Disorders of lipid metabolism                54.9865  55.8904  55.4348   
Essential hypertension                       65.0259  49.8016  56.4045   
Fluid and electrolyte disorders              48.6922  43.2916  45.8333   
Gastrointestinal hemorrhage           

Running inference: 100%|██████████| 45/45 [00:02<00:00, 21.78it/s]


                                           precision   recall       f1  \
Acute and unspecified renal failure          47.7509  31.7972  38.1743   
Acute cerebrovascular disease                 0.0000   0.0000   0.0000   
Acute myocardial infarction                  33.3333   2.1053   3.9604   
Cardiac dysrhythmias                         69.8292  67.1533  68.4651   
Chronic kidney disease                       65.8854  74.4118  69.8895   
Chronic obstructive pulmonary disease        54.3860  49.4681  51.8106   
Conduction disorders                          0.0000   0.0000   0.0000   
Congestive heart failure; nonhypertensive    74.4505  50.1852  59.9558   
Coronary atherosclerosis and related         55.3444  67.9300  60.9948   
Disorders of lipid metabolism                55.9633  64.0420  59.7307   
Essential hypertension                       59.8000  58.0583  58.9163   
Fluid and electrolyte disorders              50.4630  37.9791  43.3400   
Gastrointestinal hemorrhage           

Running inference: 100%|██████████| 45/45 [00:02<00:00, 21.51it/s]

                                           precision   recall       f1  \
Acute and unspecified renal failure          45.8904  31.3817  37.2740   
Acute cerebrovascular disease                 0.0000   0.0000   0.0000   
Acute myocardial infarction                  33.3333   2.0408   3.8462   
Cardiac dysrhythmias                         72.2581  62.9213  67.2673   
Chronic kidney disease                       69.9708  72.9483  71.4286   
Chronic obstructive pulmonary disease        50.0000  44.7059  47.2050   
Conduction disorders                          0.0000   0.0000   0.0000   
Congestive heart failure; nonhypertensive    75.3943  44.5896  56.0375   
Coronary atherosclerosis and related         57.9897  70.0935  63.4697   
Disorders of lipid metabolism                54.1766  62.1918  57.9082   
Essential hypertension                       63.3803  62.5000  62.9371   
Fluid and electrolyte disorders              50.1247  35.9571  41.8750   
Gastrointestinal hemorrhage           

In [18]:
def topk_avg_performance_formatted(performances, long_seq_performances, k=5):
    metrics = ["f1", "auc", "prauc"]
    scores = {m: np.array([p[m] for p in performances]) for m in metrics}

    # 计算排名（值越大排名越靠前）
    ranks = {m: (-scores[m]).argsort().argsort() + 1 for m in metrics}
    avg_ranks = np.mean(np.stack([ranks[m] for m in metrics], axis=1), axis=1)

    # 选 top-k
    topk_idx = np.argsort(avg_ranks)[:k]
    final_avg = {m: np.mean([performances[i][m] for i in topk_idx]) for m in performances[0].keys()}
    final_std = {m: np.std([performances[i][m] for i in topk_idx], ddof=0) for m in performances[0].keys()}
    final_long_seq_avg = {m: np.mean([long_seq_performances[i][m] for i in topk_idx]) for m in long_seq_performances[0].keys()}
    final_long_seq_std = {m: np.std([long_seq_performances[i][m] for i in topk_idx], ddof=0) for m in long_seq_performances[0].keys()}

    # 打印结果（转百分比，均保留两位小数）
    print("Final Metrics:")
    for m in performances[0].keys():
        mean_val = final_avg[m]
        std_val = final_std[m]
        print(f"{m}: {mean_val:.2f} ± {std_val:.2f}")
    print("\nFinal Long Sequence Metrics:")
    for m in long_seq_performances[0].keys():
        mean_val = final_long_seq_avg[m]
        std_val = final_long_seq_std[m]
        print(f"{m}: {mean_val:.2f} ± {std_val:.2f}")

In [19]:
topk_avg_performance_formatted(final_metrics, final_long_seq_metrics)

Final Metrics:
precision: 41.96 ± 0.85
recall: 37.47 ± 0.94
f1: 38.39 ± 0.29
auc: 74.72 ± 0.71
prauc: 43.72 ± 0.59

Final Long Sequence Metrics:
precision: 42.92 ± 0.87
recall: 39.13 ± 0.83
f1: 40.47 ± 0.55
auc: 75.95 ± 0.55
prauc: 48.78 ± 0.65
